In [138]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List
from os.path import isfile 

In [173]:
class Transaction():
    def __init__(self) -> None:
        self.files = ["usage.xlsx","subscription.xlsx","development.xlsx","maint.xlsx","customization.xlsx"]
        self.SUBSCRIPTION_STRING = "Subsciption Fees"
        self.USAGE_STRING = "Usage fees"
        self.DEVELOPMENT_STRING = "Development fees"
        self.MAINT_STRING = "Recurring maint fees"
        self.CUSTOMISATION_STRING = "Customization"
    
    def get_dataframe(self, filename:str) -> pd.DataFrame:
        df = pd.read_excel(io=filename)
        df = df.drop(df.columns[[i for i in range(1,6)] + [8]], axis=1)
        df.columns = ['name', 'debit', 'credit']
        return df
    
    def get_transaction(self,filename:str) -> List:
        if not isfile(filename):
            return
        transactions:List = []
        entity:str = ""
        credit:float = 0
        debit:float = 0
        account_type:str = self.get_account_type(filename)
        df = self.get_dataframe(filename)
        
        for index, row in df.iterrows():
            if pd.isnull(row[0]):
                credit:float = 0
                debit:float = 0
                continue
            
            if type(row[0]) == datetime:
                credit += row["credit"]
                debit += row["debit"]
                continue
            
            string:str = row[0]
            if "Total" in string:
                string = string.replace("Total", "").strip()
                # Reset all counters and prepare for next entity
                transactions.append([f"\"{string}\"", account_type, credit-debit, credit, debit])
                credit:float = 0
                debit:float = 0
                continue
        return transactions

    def get_account_type(self,filename:str) -> str:
        if filename == "usage.xlsx":
            return self.USAGE_STRING
        elif filename == "subscription.xlsx":
            return  self.SUBSCRIPTION_STRING
        elif filename == "development.xlsx":
            return  self.DEVELOPMENT_STRING
        elif filename == "maint.xlsx":
            return  self.MAINT_STRING
        elif filename == "customization.xlsx":
            return  self.CUSTOMISATION_STRING
        return ""        

    def get_transactions(self):
        transactions:List = []
        for file in self.files:
            # store transactions
            try:
                print(file)
                transactions.extend(self.get_transaction(file))
            except:
                pass
        self.save_as_csv(transactions)

    def save_as_csv(self,list_item:List) -> None:
        np.savetxt("compiled.csv", 
           list_item,
           delimiter =", ", 
           fmt ='%s')
    
    def compile_transactions(self):
        self.get_transactions()

In [174]:
t = Transaction()
t. compile_transactions()

usage.xlsx
subscription.xlsx
development.xlsx
maint.xlsx
customization.xlsx


In [12]:
filename = "C:/Users/ngtze/Github/Blockchef-Recommender/use.xlsx"
import os
os.chdir("C:")
os.path.isfile(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'C:'